In [ ]:
import socket
import json
import time
import threading
from urllib.parse import urlparse, parse_qs
import pandas as pd

# === 配置参数 ===
PORT = 8011
HOST = "192.168.50.253"

# === 导入并启动 http_server2.py 中的 run_server_select() ===
import http_server3  # 你需要确保 http_server2.py 在同一目录或 PYTHONPATH 中

def start_server(PORT):
    http_server3.run_server(PORT)

# # === 启动 server in background thread ===
# threading.Thread(target=start_server, args=(PORT,), daemon=True).start()
# print("[*] HTTP server started on port 8011")
# time.sleep(1)  # 等待服务器启动







# === 测试函数 ===
def send_request(path, host, port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client:
        client.connect((host, port))
        request = f"GET {path} HTTP/1.0\r\nHost: {host}\r\n\r\n"
        client.sendall(request.encode())
        response = b""
        while True:
            data = client.recv(4096)
            if not data:
                break
            response += data
    return response.decode()


# === 测试用例 ===
test_cases = [
    {
        "description": "Valid multiplication",
        "path": "/product?a=12&b=60&another=0.5",
        "expected_status": "200 OK",
        "expected_result": {
            "operation": "product",
            "operands": [12.0, 60.0, 0.5],
            "result": 360.0
        }
    },
    {
        "description": "Empty query string",
        "path": "/product",
        "expected_status": "400 Bad Request",
        "expected_result": {"error": "No operands"}
    },
    {
        "description": "Invalid parameter value",
        "path": "/product?a=foo",
        "expected_status": "400 Bad Request",
        "expected_result": {"error": "Invalid operand"}
    },
    {
        "description": "Wrong path",
        "path": "/wrongpath?a=1",
        "expected_status": "404 Not Found",
        "expected_result": "404 Not Found"
    },
    {
        "description": "Floating-point overflow",
        "path": f"/product?a={1e308}&b=1e10",
        "expected_status": "200 OK",
        "expected_result": {
            "operation": "product",
            "operands": [1e+308, 1e+10],
            "result": "inf"
        }
    }
]



# === 调用 server 函数（可调试） ===
print(f"[i] Calling run_server({PORT}) from Jupyter...")
start_server(PORT)

# === 等待 server 启动 ===
time.sleep(1)

# # === 执行测试 ===
# # ✅ 1. 正确的乘法请求（应返回 200 OK）
# curl "http://192.168.50.253:8011/product?a=12&b=60&another=0.5"

# # ❌ 2. 空参数（应返回 400 Bad Request）
# curl "http://192.168.50.253:8011/product"


# # ❌ 3. 非法参数（应返回 400 Bad Request）
# curl "http://192.168.50.253:8011/product?a=foo"

# # ❌ 4. 路径错误（应返回 404 Not Found）
# curl "http://192.168.50.253:8011/wrongpath?a=1"

# # ⚠️ 5. 浮点数溢出（应返回 200 OK，结果为 "inf"）
# curl "http://192.168.50.253:8011/product?a=1e308&b=1e10"






[i] Calling run_server(8011) from Jupyter...
[*] Listening on port 8011...


In [ ]:
curl "http://192.168.50.253:8011/product?a=foo"
